Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/reinforcement-learning/cartpole_on_single_compute/cartpole_sc.png)

# Reinforcement Learning in Azure Machine Learning - Cartpole Problem on Single Compute

Reinforcement Learning in Azure Machine Learning is a managed service for running reinforcement learning training and simulation. With Reinforcement Learning in Azure Machine Learning, data scientists can start developing reinforcement learning systems on one machine, and scale to compute targets with 100s of nodes if needed.

This example shows how to use Reinforcement Learning in Azure Machine Learning to train a Cartpole playing agent on a single compute. 

### Cartpole problem

Cartpole, also known as [Inverted Pendulum](https://en.wikipedia.org/wiki/Inverted_pendulum), is a pendulum with a center of mass above its pivot point. This formation is essentially unstable and will easily fall over but can be kept balanced by applying appropriate horizontal forces to the pivot point.

<table style="width:50%">
  <tr>
    <th>
      <img src="./images/cartpole.png" alt="Cartpole image" /> 
    </th>
  </tr>
  <tr>
      <th><p>Fig 1. Cartpole problem schematic description (from <a href="https://towardsdatascience.com/cartpole-introduction-to-reinforcement-learning-ed0eb5b58288">towardsdatascience.com</a>).</p></th>
  </tr>
</table>

The goal here is to train an agent to keep the cartpole balanced by applying appropriate forces to the pivot point.

See [this video](https://www.youtube.com/watch?v=XiigTGKZfks) for a real-world demonstration of cartpole problem.

### Prerequisite
The user should have completed the Azure Machine Learning Tutorial: [Get started creating your first ML experiment with the Python SDK](https://docs.microsoft.com/en-us/azure/machine-learning/tutorial-1st-experiment-sdk-setup). You will need to make sure that you have a valid subscription ID, a resource group, and an Azure Machine Learning workspace. All datastores and datasets you use should be associated with your workspace.

## Set up Development Environment
The following subsections show typical steps to setup your development environment. Setup includes:

* Connecting to a workspace to enable communication between your local machine and remote resources
* Creating an experiment to track all your runs
* Creating a remote compute target to use for training

### Azure Machine Learning SDK 
Display the Azure Machine Learning SDK version.

In [ ]:
import azureml.core

print("Azure Machine Learning SDK version:", azureml.core.VERSION)

### Get Azure Machine Learning workspace
Get a reference to an existing Azure Machine Learning workspace.

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, sep = ' | ')

### Create a new compute resource or attach an existing one

A compute target is a designated compute resource where you run your training and simulation scripts. This location may be your local machine or a cloud-based compute resource. The code below shows how to create a cloud-based compute target. For more information see [What are compute targets in Azure Machine Learning?](https://docs.microsoft.com/en-us/azure/machine-learning/concept-compute-target)

> Note that if you have an AzureML Data Scientist role, you will not have permission to create compute resources. Talk to your workspace or IT admin to create the compute targets described in this section, if they do not already exist.

**Note: Creation of a compute resource can take several minutes**. Please make sure to change `STANDARD_D2_V2` to a [size available in your region](https://azure.microsoft.com/en-us/global-infrastructure/services/?products=virtual-machines).

In [ ]:
from azureml.core.compute import AmlCompute, ComputeTarget
import os

# Choose a name and maximum size for your cluster
compute_name = "cpu-cluster-d2"
compute_min_nodes = 0
compute_max_nodes = 4
vm_size = "STANDARD_D2_V2"

if compute_name in ws.compute_targets:
    print("Found an existing compute target of name: " + compute_name)
    compute_target = ws.compute_targets[compute_name]
    # Note: you may want to make sure compute_target is of type AmlCompute        
else:
    print("Creating new compute target...")
    provisioning_config = AmlCompute.provisioning_configuration(
        vm_size=vm_size,
        min_nodes=compute_min_nodes, 
        max_nodes=compute_max_nodes)
        
    # Create the cluster
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

print(compute_target.get_status().serialize())

### Create Azure Machine Learning experiment
Create an experiment to track the runs in your workspace. 

In [ ]:
from azureml.core.experiment import Experiment

experiment_name = 'CartPole-v0-SC'
experiment = Experiment(workspace=ws, name=experiment_name)

In [ ]:
from azureml.core import Environment
import os

ray_environment_name = 'cartpole-ray-sc'
ray_environment_dockerfile_path = os.path.join(os.getcwd(), 'files', 'docker', 'Dockerfile')

# Build environment image
ray_environment = Environment. \
    from_dockerfile(name=ray_environment_name, dockerfile=ray_environment_dockerfile_path). \
    register(workspace=ws)
ray_env_build_details = ray_environment.build(workspace=ws)

ray_env_build_details.wait_for_completion(show_output=True)

## Train Cartpole Agent
In this section, we show how to use Azure Machine Learning jobs and Ray/RLlib framework to train a cartpole playing agent. 

### Create reinforcement learning training run

The code below submits the training run using a `ScriptRunConfig`. By providing the
command to run the training, and a `RunConfig` object configured with your
compute target, number of nodes, and environment image to use.

In [ ]:
from azureml.core import RunConfiguration, ScriptRunConfig, Experiment
from azureml.core.runconfig import DockerConfiguration, RunConfiguration

training_algorithm = "PPO"
rl_environment = "CartPole-v0"
video_capture = True
if video_capture:
    algorithm_config = '\'{"num_gpus": 0, "num_workers": 1, "monitor": true}\''
else:
    algorithm_config = '\'{"num_gpus": 0, "num_workers": 1, "monitor": false}\''

script_name = 'cartpole_training.py'
script_arguments = [
    '--run', training_algorithm,
    '--env', rl_environment,
    '--stop', '\'{"episode_reward_mean": 200, "time_total_s": 300}\'',
    '--config', algorithm_config,
    '--checkpoint-freq', '2',
    '--checkpoint-at-end',
    '--local-dir', './logs'
]

ray_environment = Environment.get(ws, name=ray_environment_name)
run_config = RunConfiguration(communicator='OpenMpi')
run_config.target = compute_target
run_config.node_count = 1
run_config.environment = ray_environment
command=["python", script_name, *script_arguments]

if video_capture:
    command = ["xvfb-run -s '-screen 0 640x480x16 -ac +extension GLX +render' "] + command
    run_config.environment_variables["SDL_VIDEODRIVER"] = "dummy"

training_config = ScriptRunConfig(source_directory='./files',
            command=command,
            run_config = run_config
            )

training_run = experiment.submit(training_config)

### Training script

As recommended in RLlib documentations, we use Ray Tune API to run the training algorithm. All the RLlib built-in trainers are compatible with the Tune API. Here we use `tune.run()` to execute a built-in training algorithm. For convenience, down below you can see part of the entry script where we make this call.

This is the list of parameters we are passing into `tune.run()` via the `script_params` parameter:

- `run_or_experiment`: name of the [built-in algorithm](https://ray.readthedocs.io/en/latest/rllib-algorithms.html#rllib-algorithms), 'PPO' in our example,
- `config`: Algorithm-specific configuration. This includes specifying the environment, `env`, which in our example is the gym **[CartPole-v0](https://www.gymlibrary.dev/environments/classic_control/cart_pole/)** environment,
- `stop`: stopping conditions, which could be any of the metrics returned by the trainer. Here we use "mean of episode reward", and "total training time in seconds" as stop conditions, and
- `checkpoint_freq` and `checkpoint_at_end`: Frequency of taking checkpoints (number of training iterations between checkpoints), and if a checkpoint should be taken at the end.

We also specify the `local_dir`, the directory in which the training logs, checkpoints and other training artificats will be recorded. 

See [RLlib Training APIs](https://ray.readthedocs.io/en/latest/rllib-training.html#rllib-training-apis) for more details, and also [Training (tune.run, tune.Experiment)](https://ray.readthedocs.io/en/latest/tune/api_docs/execution.html#training-tune-run-tune-experiment) for the complete list of parameters.

```python
import ray
import ray.tune as tune

if __name__ == "__main__":

    # parse arguments ...
    
    # Start ray head (single node)
    os.system('ray start --head')
    ray.init(address='auto')

    # Run training task using tune.run
    tune.run(
        run_or_experiment=args.run,
        config=dict(args.config, env=args.env),
        stop=args.stop,
        checkpoint_freq=args.checkpoint_freq,
        checkpoint_at_end=args.checkpoint_at_end,
        local_dir=args.local_dir
    )
```

### Monitor experiment

Azure Machine Learning provides a Jupyter widget to show the status of an experiment run. You could use this widget to monitor the status of the runs.

In [ ]:
from azureml.widgets import RunDetails

RunDetails(training_run).show()

### Stop the run
To stop the run, call `training_run.cancel()`.

In [ ]:
# Uncomment line below to cancel the run
# training_run.cancel()

### Wait for completion
Wait for the run to complete before proceeding.

**Note: The length of the run depends on the provisioning time of the compute target and it may take several minutes to complete.**

In [ ]:
training_run.wait_for_completion()

### Get access to training artifacts
We can simply use run id to get a handle to an in-progress or a previously concluded run.

In [ ]:
from azureml.core import Run

run_id = training_run.id # Or set to run id of a completed run (e.g. 'rl-cartpole-v0_1587572312_06e04ace_head')
run = Run(experiment, run_id=run_id)

Now we can use the Run API to download policy training artifacts (saved model and checkpoints) to local compute.

In [ ]:
from os import path
from distutils import dir_util

training_artifacts_path = path.join("logs", training_algorithm)
print("Training artifacts path:", training_artifacts_path)

if path.exists(training_artifacts_path):
    dir_util.remove_tree(training_artifacts_path)

# Download run artifacts to local compute
training_run.download_files(training_artifacts_path)

### Display movies of selected training episodes

Ray creates video output of selected training episodes in mp4 format.  Here we will display two of these, i.e. the first and the last recorded videos, so you could see the improvement of the agent after training.

First we introduce a few helper functions: a function to download the movies from our dataset, another one to find mp4 movies in a local directory, and one more to display a downloaded movie.

In [ ]:
import shutil

# A helper function to find movies in a directory
def find_movies(movie_path):
    print("Looking in path:", movie_path)
    mp4_movies = []
    for root, _, files in os.walk(movie_path):
        for name in files:
            if name.endswith('.mp4'):
                mp4_movies.append(path.join(root, name))
    print('Found {} movies'.format(len(mp4_movies)))

    return mp4_movies


# A helper function to display a movie
from IPython.core.display import Video
from IPython.display import display
def display_movie(movie_file):
    display(Video(movie_file, embed=True, html_attributes='controls'))

Look for the downloaded movies in the local directory and sort them.

In [ ]:
mp4_files = find_movies(training_artifacts_path)
mp4_files.sort()

Display a movie of the first training episode.  This is how the agent performs with no training.

In [ ]:
first_movie = mp4_files[0] if len(mp4_files) > 0 else None
print("First movie:", first_movie)

display_movie(first_movie)

Display a movie of the last training episode.  This is how a fully-trained agent performs.

In [ ]:
last_movie = mp4_files[-1] if len(mp4_files) > 0 else None
print("Last movie:", last_movie)

display_movie(last_movie)

## Evaluate Trained Agent and See Results

We can evaluate a previously trained policy using the `rollout.py` helper script provided by RLlib (see [Evaluating Trained Policies](https://ray.readthedocs.io/en/latest/rllib-training.html#evaluating-trained-policies) for more details). Here we use an adaptation of this script to reconstruct a policy from a checkpoint taken and saved during training. We took these checkpoints by setting `checkpoint-freq` and `checkpoint-at-end` parameters above.
In this section we show how to use these checkpoints to evaluate the trained policy.

### Evaluate a trained policy
In this section, we submit another job, to evalute a trained policy. The entrypoint for this job is
`cartpole-rollout.py` script, and we we pass the checkpoints dataset to this script as a dataset refrence.

We are using script parameters to pass in the same algorithm and the same environment used during training. We also specify the checkpoint number of the checkpoint we wish to evaluate, `checkpoint-number`, and number of the steps we shall run the rollout, `steps`.

The training artifacts dataset will be accessible to the rollout script as a mounted folder. The mounted folder and the checkpoint number, passed in via `checkpoint-number`, will be used to create a path to the checkpoint we are going to evaluate. The created checkpoint path then will be passed into RLlib rollout script for evaluation.

Let's find the checkpoints and the last checkpoint number first.

In [ ]:
# A helper function to find checkpoint files in a directory
def find_checkpoints(file_path):
    print("Looking in path:", file_path)
    checkpoints = []
    for root, _, files in os.walk(file_path):
        for name in files:
            if os.path.basename(root).startswith('checkpoint_'):
                checkpoints.append(path.join(root, name))
    return checkpoints

checkpoint_files = find_checkpoints(training_artifacts_path)

In [ ]:
# Find checkpoints and last checkpoint number
checkpoint_numbers = []
for file in checkpoint_files:
    file = os.path.basename(file)
    if file.startswith('checkpoint-') and not file.endswith('.tune_metadata'):
        checkpoint_numbers.append(int(file.split('-')[-1]))

print("Checkpoints:", checkpoint_numbers)

last_checkpoint_number = max(checkpoint_numbers)
print("Last checkpoint number:", last_checkpoint_number)

In [ ]:
# Upload the checkpoint files and create a DataSet
from azureml.core import Dataset

datastore = ws.get_default_datastore()
checkpoint_dataref = datastore.upload_files(checkpoint_files, target_path='cartpole_checkpoints_' + run_id, overwrite=True)
checkpoint_ds = Dataset.File.from_files(checkpoint_dataref)

You can submit the training run using a `ScriptRunConfig`. By providing the
command to run the training, and a `RunConfig` object configured w

In [ ]:
ray_environment_name = 'cartpole-ray-sc'

experiment_name = 'CartPole-v0-SC'
training_algorithm = 'PPO'
rl_environment = 'CartPole-v0'

experiment = Experiment(workspace=ws, name=experiment_name)
ray_environment = Environment.get(workspace=ws, name=ray_environment_name)

script_name = 'cartpole_rollout.py'
video_capture = True
if video_capture:
    script_arguments = ['--video-dir', './logs/video']
else:
    script_arguments = ['--no-render']
script_arguments = script_arguments + [
    '--run', training_algorithm,
    '--env', rl_environment,
    '--config', '{}',
    '--steps', '2000',
    '--checkpoint-number', str(last_checkpoint_number),
    '--artifacts-dataset', checkpoint_ds.as_named_input('artifacts_dataset'),
    '--artifacts-path', checkpoint_ds.as_named_input('artifacts_path').as_mount()
]

command = ["python", script_name, *script_arguments]

if video_capture:
    command = ["xvfb-run -s '-screen 0 640x480x16 -ac +extension GLX +render' "] + command
    run_config.environment_variables["SDL_VIDEODRIVER"] = "dummy"

run_config = RunConfiguration(communicator='OpenMpi')
run_config.target = compute_target
run_config.docker = DockerConfiguration(use_docker=True)
run_config.node_count = 1
run_config.environment = ray_environment

rollout_config = ScriptRunConfig(
                    source_directory='./files',
                    command=command,
                    run_config=run_config
                   )

rollout_run = experiment.submit(rollout_config)
rollout_run

And then, similar to the training section, we can monitor the real-time progress of the rollout run and its chid as follows. If you browse logs of the child run you can see the evaluation results recorded in driver_log.txt file. Note that you may need to wait several minutes before these results become available.

In [ ]:
RunDetails(rollout_run).show()

Wait for completion of the rollout run before moving to the next section, or you may cancel the run.

In [ ]:
# Uncomment line below to cancel the run
#rollout_run.cancel()
rollout_run.wait_for_completion()

### Display movies of selected rollout episodes

To display recorded movies first we download recorded videos to local machine. Here again we create a dataset of rollout artifacts and use the helper functions introduced above to download and displays rollout videos.

In [ ]:
# Download rollout artifacts
rollout_artifacts_path = path.join("logs", "rollout")
print("Rollout artifacts path:", rollout_artifacts_path)

if path.exists(rollout_artifacts_path):
    dir_util.remove_tree(rollout_artifacts_path)

# Download videos to local compute
rollout_run.download_files("logs/video", output_directory = rollout_artifacts_path)

Now, similar to the training section, we look for the last video.

In [ ]:
# Look for the downloaded movie in local directory
mp4_files = find_movies(rollout_artifacts_path)
mp4_files.sort()
last_movie = mp4_files[-1] if len(mp4_files) > 1 else None
print("Last movie:", last_movie)

Display last video recorded during the rollout.

In [ ]:
last_movie = mp4_files[-1] if len(mp4_files) > 0 else None
print("Last movie:", last_movie)

display_movie(last_movie)

## Cleaning up
For your convenience, below you can find code snippets to clean up any resources created as part of this tutorial that you don't wish to retain.

In [ ]:
# To archive the created experiment:
#exp.archive()

# To delete the compute target:
#compute_target.delete()

# To delete downloaded training artifacts
#if os.path.exists(training_artifacts_path):
#    dir_util.remove_tree(training_artifacts_path)

# To delete downloaded rollout videos
#if path.exists(rollout_artifacts_path):
#    dir_util.remove_tree(rollout_artifacts_path)

## Next
This example was about running Reinforcement Learning in Azure Machine Learning (Ray/RLlib Framework) on a single compute. Please see [Pong Problem](../atari-on-distributed-compute/pong_rllib.ipynb)
example which uses Ray RLlib to train a Pong playing agent on a multi-node cluster.